In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from os.path import join
from tqdm import tqdm
import random
from matplotlib import pyplot as plt
import math

project_root = Path('..')

preprocess_path = join(project_root, Path('data/preprocess'))
random.seed(10)

Thoughts: thresholds need to be custom to each profile. This would be a good opportunity to try to make a profile that gets more confident the more data about the author it has. The profile could model distributions of the differences between its sentences to its mean and other people's sentences to its mean. Then it can use these two distributions to determine which is more likely for incoming sentences. 

Question: Can euclidean distances from the mean or cosine similarites be treated as normal random variables?

In [2]:
# Using function words for these experiments
function_words_train = pd.read_hdf(join(preprocess_path, "bawe_train_preprocessed_function_word_counter.hdf5"))
pos_bigrams_train = pd.read_hdf(join(preprocess_path, "bawe_train_preprocessed_pos2gram_counter.hdf5"))
train_set = pd.read_hdf(join(preprocess_path, "bawe_train_sentences.hdf5"))
# function_words_train = pd.concat([function_words_train, pos_bigrams_train], axis=1)
function_words_train = pos_bigrams_train

function_words_train

0    1    2    3    4    5    6    7    8    \
author text_id sentence_position                                                
1      0       0                  1.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0   
               1                  0.0  1.0  0.0  0.0  0.0  0.0  0.0  4.0  0.0   
               2                  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   
               3                  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   
               4                  0.0  0.0  0.0  0.0  2.0  0.0  0.0  4.0  0.0   
...                               ...  ...  ...  ...  ...  ...  ...  ...  ...   
6998   2       175                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               176                0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0   
               177                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               178                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               179                0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0  0.0   

                                  9    ...  314  315  316  317  318  319  320  \
author text_id sentence_position       ...                                      
1      0       0                  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               1                  1.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               2                  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               3                  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               4                  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                               ...  ...  ...  ...  ...  ...  ...  ...  ...   
6998   2       175                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               176                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               177                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               178                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               179                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                  321  322  323  
author text_id sentence_position                 
1      0       0                  0.0  0.0  0.0  
               1                  0.0  0.0  0.0  
               2                  0.0  0.0  0.0  
               3                  0.0  0.0  0.0  
               4                  0.0  0.0  0.0  
...                               ...  ...  ...  
6998   2       175                0.0  0.0  0.0  
               176                0.0  0.0  0.0  
               177                0.0  0.0  0.0  
               178                0.0  0.0  0.0  
               179                0.0  0.0  0.0  

[238088 rows x 324 columns]

In [3]:
std_train = (function_words_train - function_words_train.mean()) / function_words_train.std()
std_train

0         1         2         3    \
author text_id sentence_position                                           
1      0       0                  1.799867 -0.374486 -0.113872 -0.086894   
               1                 -0.308537  2.114088 -0.113872 -0.086894   
               2                 -0.308537 -0.374486 -0.113872 -0.086894   
               3                 -0.308537 -0.374486 -0.113872 -0.086894   
               4                 -0.308537 -0.374486 -0.113872 -0.086894   
...                                    ...       ...       ...       ...   
6998   2       175               -0.308537 -0.374486 -0.113872 -0.086894   
               176               -0.308537 -0.374486 -0.113872 -0.086894   
               177               -0.308537 -0.374486 -0.113872 -0.086894   
               178               -0.308537 -0.374486 -0.113872 -0.086894   
               179               -0.308537 -0.374486 -0.113872 -0.086894   

                                       4         5         6         7    \
author text_id sentence_position                                           
1      0       0                 -0.262186 -0.083779 -0.010276  0.600482   
               1                 -0.262186 -0.083779 -0.010276  2.120695   
               2                 -0.262186 -0.083779 -0.010276 -0.159624   
               3                 -0.262186 -0.083779 -0.010276 -0.159624   
               4                  6.838644 -0.083779 -0.010276  2.120695   
...                                    ...       ...       ...       ...   
6998   2       175               -0.262186 -0.083779 -0.010276 -0.919730   
               176               -0.262186 -0.083779 -0.010276 -0.159624   
               177               -0.262186 -0.083779 -0.010276 -0.919730   
               178               -0.262186 -0.083779 -0.010276 -0.919730   
               179               -0.262186 -0.083779 -0.010276  0.600482   

                                       8         9    ...       314       315  \
author text_id sentence_position                      ...                       
1      0       0                 -0.097418 -0.219697  ... -0.053934 -0.021452   
               1                 -0.097418  4.110734  ... -0.053934 -0.021452   
               2                 -0.097418 -0.219697  ... -0.053934 -0.021452   
               3                 -0.097418 -0.219697  ... -0.053934 -0.021452   
               4                 -0.097418 -0.219697  ... -0.053934 -0.021452   
...                                    ...       ...  ...       ...       ...   
6998   2       175               -0.097418 -0.219697  ... -0.053934 -0.021452   
               176               -0.097418 -0.219697  ... -0.053934 -0.021452   
               177               -0.097418 -0.219697  ... -0.053934 -0.021452   
               178               -0.097418 -0.219697  ... -0.053934 -0.021452   
               179               -0.097418 -0.219697  ... -0.053934 -0.021452   

                                       316       317       318       319  \
author text_id sentence_position                                           
1      0       0                 -0.048032 -0.227031 -0.170571 -0.028995   
               1                 -0.048032 -0.227031 -0.170571 -0.028995   
               2                 -0.048032 -0.227031 -0.170571 -0.028995   
               3                 -0.048032 -0.227031 -0.170571 -0.028995   
               4                 -0.048032 -0.227031 -0.170571 -0.028995   
...                                    ...       ...       ...       ...   
6998   2       175               -0.048032 -0.227031 -0.170571 -0.028995   
               176               -0.048032 -0.227031 -0.170571 -0.028995   
               177               -0.048032 -0.227031 -0.170571 -0.028995   
               178               -0.048032 -0.227031 -0.170571 -0.028995   
               179               -0.048032 -0.227031 -0.170571 -0.028995   

                  

In [ ]:
pca_vals, pca_vecs = np.linalg.eig(std_train.cov())

pca_vals

In [3]:
# train_set.loc[(2, 3)]
train_set.loc[6]

sentence
text_id sentence_position                                                   
0       0                        <abstract><heading>Aim / Abstract</heading>
        1                  The aim of the experiment was to establish the...
        2                  Using conjugation, pieces of Escherichia coli ...
        3                  As this occurs in a linear fashion in one dire...
        4                  The plates were incubated for a week and from ...
...                                                                      ...
2       107                                       <heading>Summary</heading>
        108                The evidence above strongly suggests that orga...
        109                As the host eukaryotic cell and symbionts deve...
        110                As Margulis and Sagan said one of their books,...
        111                                                                8

[253 rows x 1 columns]

In [4]:
def select_good_features(df):
    overall_var = df.var()

    author_vars = df.groupby(level="author").var()

    mean_explained_var = (overall_var - author_vars).mean()

    # Features that reduce the variance within classes should hopefully be good
    # features.
    selections = mean_explained_var > 0

    # The index of selctions should be the columns of the dataframe given the
    # last few operations.
    chosen_columns = selections[selections].index.tolist()

    return df[chosen_columns]

In [5]:
filtered_train = select_good_features(function_words_train)

filtered_train

0    6    7    11   14   17   18   23   28   \
author text_id sentence_position                                                
1      0       0                  1.0  0.0  2.0  0.0  0.0  0.0  1.0  0.0  1.0   
               1                  0.0  0.0  4.0  0.0  0.0  0.0  1.0  1.0  0.0   
               2                  0.0  0.0  1.0  0.0  0.0  0.0  0.0  3.0  0.0   
               3                  0.0  0.0  1.0  0.0  0.0  0.0  1.0  2.0  0.0   
               4                  0.0  0.0  4.0  0.0  0.0  0.0  1.0  3.0  1.0   
...                               ...  ...  ...  ...  ...  ...  ...  ...  ...   
6998   2       175                0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0  1.0   
               176                0.0  0.0  1.0  0.0  0.0  0.0  0.0  2.0  0.0   
               177                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               178                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               179                0.0  0.0  2.0  0.0  0.0  0.0  0.0  4.0  0.0   

                                  29   ...  309  311  312  315  316  317  318  \
author text_id sentence_position       ...                                      
1      0       0                  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               1                  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               2                  1.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               3                  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               4                  0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                               ...  ...  ...  ...  ...  ...  ...  ...  ...   
6998   2       175                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               176                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               177                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               178                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
               179                0.0  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                                  319  322  323  
author text_id sentence_position                 
1      0       0                  0.0  0.0  0.0  
               1                  0.0  0.0  0.0  
               2                  0.0  0.0  0.0  
               3                  0.0  0.0  0.0  
               4                  0.0  0.0  0.0  
...                               ...  ...  ...  
6998   2       175                0.0  0.0  0.0  
               176                0.0  0.0  0.0  
               177                0.0  0.0  0.0  
               178                0.0  0.0  0.0  
               179                0.0  0.0  0.0  

[238088 rows x 128 columns]

In [6]:
authors = filtered_train.index.get_level_values("author")
author_set = list(set(authors))

# experiment_authors = random.sample(author_set, 5)
experiment_authors = [1, 2, 5, 6]

experiment_authors

[1, 2, 5, 6]

In [7]:
# chosen_author = experiment_authors[5]
chosen_author = 6

chosen_author_sentences = filtered_train.loc[chosen_author]

chosen_author_sentences

0    6    7    11   14   17   18   23   28   29   \
text_id sentence_position                                                     
0       0                  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        1                  1.0  0.0  1.0  0.0  0.0  0.0  1.0  2.0  0.0  0.0   
        2                  0.0  0.0  1.0  0.0  0.0  0.0  0.0  2.0  0.0  1.0   
        3                  0.0  0.0  1.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0   
        4                  0.0  0.0  2.0  0.0  0.0  0.0  0.0  5.0  0.0  0.0   
...                        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2       107                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        108                0.0  0.0  2.0  0.0  0.0  0.0  2.0  1.0  0.0  0.0   
        109                0.0  0.0  3.0  0.0  0.0  0.0  0.0  3.0  1.0  0.0   
        110                0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0   
        111                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                           ...  309  311  312  315  316  317  318  319  322  \
text_id sentence_position  ...                                                
0       0                  ...  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   
        1                  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        2                  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        3                  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        4                  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2       107                ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0   
        108                ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        109                ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        110                ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        111                ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                           323  
text_id sentence_position       
0       0                  0.0  
        1                  0.0  
        2                  0.0  
        3                  0.0  
        4                  0.0  
...                        ...  
2       107                0.0  
        108                0.0  
        109                0.0  
        110                0.0  
        111                0.0  

[253 rows x 128 columns]

Question: If I take the mean of each sentence distance and include it in the mean instead of excluding it, does this affect the mean/var distance?

In [8]:
included_sentences_distances = np.linalg.norm(chosen_author_sentences - chosen_author_sentences.mean(), axis=1)

included_sentences_distances_mean = included_sentences_distances.mean()
included_sentences_distances_var = included_sentences_distances.var()


def get_excluded_sentence_distances(array):
    def sentence_distance(i):
        selection = [True] * len(array)
        selection[i] = False
        return np.linalg.norm(array[i] - np.mean(array[selection]))

    return np.array([sentence_distance(index) for index in range(len(array))])


excluded_sentence_distances = get_excluded_sentence_distances(chosen_author_sentences.to_numpy())

excluded_sentence_distances_mean = excluded_sentence_distances.mean()
excluded_sentence_distances_var = excluded_sentence_distances.var()

included_sentences_distances_mean, excluded_sentence_distances_mean

(3.9830224162014436, 5.035381492758212)

Answer: Yes :(, the mean is consistently much lower because the sentence is included. This is unfortunate because the other method was much more efficient to compute.

In [9]:
# plt.hist(excluded_sentence_distances)

# plt.show()

Answer to previous question: Euclidean distances are not normal (no doy why would they be? euclidean distances from same distribution mean should on average be close to 0 and can't be lower than that).

Question: What if I just pick a threshold that accounts for 95% of the author's sentences.

In [10]:
# chosen_text = chosen_author_sentences.loc[0]

# chosen_sentence_distances = get_excluded_sentence_distances(chosen_text.to_numpy())

# index_threshold = math.floor(len(chosen_sentence_distances) * 0.6)
# threshold = np.sort(chosen_sentence_distances)[index_threshold]

# threshold, index_threshold

In [11]:
chosen_author_sentences

0    6    7    11   14   17   18   23   28   29   \
text_id sentence_position                                                     
0       0                  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        1                  1.0  0.0  1.0  0.0  0.0  0.0  1.0  2.0  0.0  0.0   
        2                  0.0  0.0  1.0  0.0  0.0  0.0  0.0  2.0  0.0  1.0   
        3                  0.0  0.0  1.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0   
        4                  0.0  0.0  2.0  0.0  0.0  0.0  0.0  5.0  0.0  0.0   
...                        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2       107                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        108                0.0  0.0  2.0  0.0  0.0  0.0  2.0  1.0  0.0  0.0   
        109                0.0  0.0  3.0  0.0  0.0  0.0  0.0  3.0  1.0  0.0   
        110                0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0   
        111                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                           ...  309  311  312  315  316  317  318  319  322  \
text_id sentence_position  ...                                                
0       0                  ...  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   
        1                  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        2                  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        3                  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        4                  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2       107                ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0   
        108                ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        109                ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        110                ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        111                ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                           323  
text_id sentence_position       
0       0                  0.0  
        1                  0.0  
        2                  0.0  
        3                  0.0  
        4                  0.0  
...                        ...  
2       107                0.0  
        108                0.0  
        109                0.0  
        110                0.0  
        111                0.0  

[253 rows x 128 columns]

In [12]:
chosen_text = chosen_author_sentences.drop(index=(0,))

chosen_text

0    6    7    11   14   17   18   23   28   29   \
text_id sentence_position                                                     
1       0                  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        1                  0.0  0.0  1.0  0.0  0.0  0.0  0.0  6.0  0.0  0.0   
        2                  1.0  0.0  2.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0   
        3                  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0   
        4                  0.0  0.0  1.0  0.0  0.0  0.0  1.0  1.0  0.0  0.0   
...                        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2       107                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        108                0.0  0.0  2.0  0.0  0.0  0.0  2.0  1.0  0.0  0.0   
        109                0.0  0.0  3.0  0.0  0.0  0.0  0.0  3.0  1.0  0.0   
        110                0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0   
        111                0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                           ...  309  311  312  315  316  317  318  319  322  \
text_id sentence_position  ...                                                
1       0                  ...  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  1.0   
        1                  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        2                  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        3                  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        4                  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                        ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2       107                ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0   
        108                ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        109                ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        110                ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        111                ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

                           323  
text_id sentence_position       
1       0                  0.0  
        1                  0.0  
        2                  0.0  
        3                  0.0  
        4                  0.0  
...                        ...  
2       107                0.0  
        108                0.0  
        109                0.0  
        110                0.0  
        111                0.0  

[191 rows x 128 columns]

In [13]:
normalized_text = (chosen_text - chosen_text.mean()) / chosen_text.var()
# Come back here on error

chosen_cov = chosen_text.cov()

eig_values, eig_vectors = np.linalg.eig(chosen_cov)
eig_values, eig_vectors = eig_values.real, eig_vectors.real

eig_sum = np.sum(eig_values)

k = 15

phi_list = chosen_text - chosen_text.mean()

omega_list = np.sum((phi_list[:k] * eig_vectors[:, :k].T), axis=1).to_numpy()

phi_hat = np.sum((omega_list[:k] * eig_vectors[:, :k]), axis=1)
np.flip(phi_hat)

array([ 6.38551082e-02,  1.01608725e-01,  0.00000000e+00,  0.00000000e+00,
        3.30797457e-02,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -3.06226722e-02,  2.07137174e-03, -4.05892896e-03,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -3.06226722e-02,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -2.75307702e-02,  2.88658373e-03,
       -6.96576237e-03,  0.00000000e+00, -3.54786769e-01, -1.65426655e-02,
       -7.40341675e-02, -4.25987465e-02, -2.89138758e-03, -1.52544247e-01,
        1.40106482e-01,  0.00000000e+00,  5.93718591e-02,  0.00000000e+00,
        1.67854648e-02,  

In [14]:
profile_mean = chosen_text.mean()

cutoff_texts = chosen_text

diffs = cutoff_texts - chosen_text.mean() - phi_hat

distances = np.linalg.norm(diffs, axis=1)

distances

array([ 6.4254438 , 14.2591007 ,  8.07990388,  6.26380063,  6.74143863,
        6.77110395,  5.28983103,  6.41167173,  6.83992115,  6.40111501,
        6.54869375,  6.26295297,  7.78402823,  7.39216234,  7.85467845,
        7.72239701, 11.50395048,  8.80716062,  8.16973991,  8.75430912,
        7.00902062,  6.85430244, 10.98609655,  7.01256944,  8.00860623,
        7.64573927,  8.97971275,  7.92612365,  7.53016824,  8.55916545,
        7.91543094,  8.3295978 ,  7.49586473,  6.38784256,  5.81505245,
        7.33646879,  7.17674514, 11.92361169,  8.70732237,  7.13260409,
        8.84680008,  7.16010108,  7.76137724,  8.68891029,  8.04842867,
        7.33691952,  6.26295297,  6.26295297,  8.26464841, 15.47298488,
        6.38617005,  7.41028048,  8.52098632,  7.55013037,  6.48607928,
        6.94013432,  6.38784256,  6.55670525,  6.59239066,  7.43121429,
       10.32677007,  8.42345816,  8.24872624,  6.47463428,  7.03101629,
       10.52885266,  5.8846479 ,  5.72315291, 10.03529127,  7.99

In [15]:
cutoff = np.mean(distances) + (np.std(distances) / 3.5)

cutoff

9.055540359688056

In [16]:
same_distances = distances[distances > cutoff]

In [17]:
outlier_cutoff = np.mean(same_distances) + (np.std(same_distances) / 3.5)

# np.sum(same_distances > outlier_cutoff) / distances.shape[0]
np.sum(same_distances > outlier_cutoff) / len(same_distances)

0.2641509433962264

In [18]:
filtered_train.loc[1].mean()

0      0.133641
6      0.000000
7      1.834101
11     0.004608
14     0.000000
         ...   
317    0.000000
318    0.018433
319    0.000000
322    0.000000
323    0.000000
Length: 128, dtype: float64

In [19]:
np.linalg.norm(filtered_train.loc[1].mean() - filtered_train.mean())

1.6848875174650744

In [20]:
# suspect_texts = filtered_train.loc[experiment_authors[1], 0]
suspect_texts = filtered_train.drop(index=(chosen_author,))
# suspect_texts = chosen_author_sentences.loc[1]

suspect_diffs = suspect_texts - profile_mean - phi_hat

suspect_distances = np.linalg.norm(suspect_diffs, axis=1)

first_suspect_distances = suspect_distances[suspect_distances > cutoff]

# np.sum(first_suspect_distances > outlier_cutoff) / len(suspect_distances)
np.sum(first_suspect_distances > outlier_cutoff) / len(first_suspect_distances)

0.23233137239100654

In [21]:
# def euclidean_distance(mean, df):
#     return np.linalg.norm(mean - df, axis=1)


# chosen_text_mean = chosen_text.mean()

# same_texts = chosen_author_sentences.drop(index=(0,))
# other_author_texts = filtered_train.drop(index=(chosen_author,))

# same_sentence_classifications = pd.DataFrame(
#     euclidean_distance(chosen_text_mean, same_texts) > threshold, index=same_texts.index
# )
# other_sentence_classifications = pd.DataFrame(
#     euclidean_distance(chosen_text_mean, other_author_texts) > threshold, index=other_author_texts.index
# )

# same_text_classifications = same_sentence_classifications.groupby(level=("text_id")).mean() > 0.5
# other_text_classifications = other_sentence_classifications.groupby(level=("author", "text_id")).mean() > 0.5

# same_flags = same_text_classifications.sum()
# same_length = len(same_text_classifications)
# other_flags = other_text_classifications.sum()
# other_length = len(other_text_classifications)

# tnr = (same_length - same_flags) / same_length
# tpr = other_flags / other_length

# tnr[0], tpr[0]